# Imports

In [1]:
import sys
from sklearn.metrics import classification_report, accuracy_score, f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
import numpy as np
import itertools
from itertools import product
import pprint
import sys
import functools as ft
from sklearn.utils import shuffle

In [11]:
sys.path.append('Code/')
from single_split_gridsearch import SingleSplitGridSearch

In [29]:
FATOR_TEST = 2
FATOR_TREINO = 2

features_folder = 'allfeatures/'

folds = [1, 2, 3, 4, 5]
patches = [1, 3, 5, 10]

# Loads e Predictions

In [12]:
def get_tensor(features, labels, n_patches):
  inicio= 0
  ty = []
  tx = []

  for i in range(0, len(features)+1, n_patches):
    tx.append(features[inicio: i+n_patches])
    ty.append(labels[inicio: i+n_patches])
    inicio = i+n_patches

  return tx, ty

def unpack_tensor(features, labels, n_patches):

  f = []
  l = []

  count = 0
  for i in features:
      for j in i:
        f.append(j)
        count = count + 1

  for i in labels:
      for j in i:
        l.append(j)
        
  return np.array(f), np.array(l)

def double_test(classe_alvo, train_X, train_Y, test_X, test_Y):

    """
      SubSet do Treino e Teste
    """

    # Seleciona os 5 Impostores para o treino
    impostores = list(range(1,81))
    impostores.remove(classe_alvo)
    np.random.shuffle(impostores)
    imp = impostores[:5]
    imp.append(classe_alvo)

    m = [train_Y == i for i in imp]
    mask = ft.reduce(np.logical_or, m)
    train_X = train_X[mask]
    train_Y = train_Y[mask]

    print("\n ------------ len(train_X)")
    print(len(train_X))
    print("\n ------------ train_X[:15")
    print(train_X[:15])

    # Seleciona a classe teste com 5 impostores
    m = [test_Y == i for i in imp]
    mask = ft.reduce(np.logical_or, m)
    test_X_five = test_X[mask]
    test_Y_five = test_Y[mask]

    # Seleciona a classe teste com Todos os impostores
    imp = impostores
    imp.append(classe_alvo)
    m = [test_Y == i for i in imp]
    mask = ft.reduce(np.logical_or, m)
    test_X_all = test_X[mask]
    test_Y_all = test_Y[mask]

    print("\n ------------ test_Y_five")
    print(test_Y_five)
    print("\n ------------ test_Y_all")
    print(test_Y_all)

    # tudo que não for o alvo é impostora = 2
    if (classe_alvo == 2):
      print("\n\n\n ----------------- AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAan", classe_alvo)
      train_Y[ train_Y == 1] = 3
      test_Y_five[ test_Y_five == 1] = 3
      test_Y_all[ test_Y_all == 1] = 3

      train_Y[ train_Y == classe_alvo] = 1
      test_Y_five[ test_Y_five == classe_alvo] = 1
      test_Y_all[ test_Y_all == classe_alvo] = 1
      
      train_Y[ train_Y != 1 ] = 2
      test_Y_five[ test_Y_five != 1] = 2
      test_Y_all[ test_Y_all != 1] = 2

    else:
      print("\n\n\n ----------------- BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB ", classe_alvo)

      
      train_Y[ train_Y != classe_alvo] = 2
      test_Y_five[ test_Y_five != classe_alvo] = 2
      test_Y_all[ test_Y_all != classe_alvo] = 2
      
      train_Y[ train_Y == classe_alvo] = 1
      test_Y_five[ test_Y_five == classe_alvo] = 1
      test_Y_all[ test_Y_all == classe_alvo] = 1



    print("\n ================ IMPOSTORES ")
    print("\n ------------ test_Y_five")
    print(test_Y_five)
    print("\n ------------ test_Y_all")
    print(test_Y_all)

    return train_X, train_Y, test_X_five, test_Y_five, test_X_all, test_Y_all


def load_data_shape(filename):
    with open(filename) as f:
        c = f.readline()
    c = [i for i in c.strip().split(" ")]
    return len(c)

#Divide  o arquivo, X é o dado, Y é as labels e F é o nome dos arquivos
def load_data(filename, class_idx, filename_idx, features_idxs):
    print (class_idx, filename_idx, features_idxs)

    with open(filename) as f:
        c = f.readlines()
    import random
    random.shuffle(c)
    c = np.array([np.array(i.strip().split(" ")) for i in c])

    X = c[:,features_idxs].astype(float)
    Y = c[:,class_idx]
    F = c[:,filename_idx]
    print('X: {} Y: {} F: {}'.format(X, Y, F))
    return X, Y, F

def get_final_predictions(n_patches, predicted):
    songs = int(predicted.shape[0] / n_patches)

    r = predicted.reshape((songs,n_patches))
    r = [np.unique(i, return_counts=True) for i in r]
    r = [ i[0][np.argmax(i[1])] for i in r]
    
    return r

def get_final_predictions_sumvote(n_patches, predicted):
    songs = int(predicted.shape[0] / n_patches)
    r = predicted.reshape((songs, n_patches,-1))    
    return np.argmax(np.sum(r, 1), -1) + 1

def get_labels(n_patches, labels):
    o = []
    for i in range(0,len(labels),n_patches):
        o.append(labels[i])
    return np.array(o)

def libsvm_to_numpy(folds, patches, input_proto, output_proto):

    for fold, n_patches in itertools.product(folds, patches):
        fname = input_proto % ( fold, n_patches)
        print (fname)

        #X, Y,F = load_data(fname, "Posição da classe", "Posição do file", "O resto de X")
        X, Y, F = load_data(fname, -2, -1, slice(0,load_data_shape(fname)-2))
        Y = Y.astype(int)
        #F = np.array(["%04d.bmp" % i for i in F])
        
        X_out = (output_proto % (fold, n_patches)) + "_X.npy"
        Y_out = (output_proto % (fold, n_patches)) + "_Y.npy"
        F_out = (output_proto % (fold, n_patches)) + "_F.npy"
        
        print( "\t%s\n\t%s\n\t%s" % (X_out, Y_out, F_out))

        np.save(X_out, X)
        np.save(Y_out, Y)
        np.save(F_out, F)

def uni_shuffle(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]
    

# Função principal

In [14]:
def run_experiment(validation_test_size, folds, total_patches, proto_x, proto_y):
    sujeitos = {}
    for classe_alvo in range(1,81):
      results = {}
      print("Classe: ", classe_alvo);
      for patches in total_patches:
      #for patches in [3]:
          print("Patch: ", patches)
          results[patches] = {'f1_five' : [], 'f1_all': []}
          for test_fold in folds:
              print(f"\n\n\n+--------------Patches: {patches} ---- Test_fold: {test_fold}")
              train_folds = sorted( list(set(folds) - set([test_fold])) )
              # train_folds = sorted( list(set(folds) - set([1])) )
              # > [2, 3, 4, 5]

              train_X = np.concatenate([np.load(proto_x % (f,patches)) for f in train_folds])
              train_Y = np.concatenate([ np.load(proto_y % (f,patches)) for f in train_folds ])

              test_X = np.load(proto_x % (test_fold,patches))
              test_Y = np.load(proto_y % (test_fold,patches))

              """
                  Fazer um Shuffle com TENSOR
                  [ [ 0,0,0], [0,0,0] , [1,1,1] ]
              """
              train_X, train_Y = get_tensor(train_X,train_Y ,patches)
              train_X, train_Y = shuffle(train_X, train_Y)
              train_X, train_Y = unpack_tensor(train_X, train_Y, patches)

              test_X, test_Y = get_tensor(test_X,test_Y ,patches)
              test_X, test_Y = shuffle(test_X, test_Y)
              test_X, test_Y = unpack_tensor(test_X, test_Y, patches)
              
              """
                SubSet do Treino e Teste
              """
              train_X, train_Y, test_X_five, test_Y_five, test_X_all, test_Y_all = double_test(classe_alvo, train_X, train_Y, test_X, test_Y)


              """
                Normalizar X's
              """
              ss = StandardScaler()
              train_X = ss.fit_transform(train_X)

              test_X_five = ss.transform(test_X_five)
              test_X_all = ss.transform(test_X_all)

              """
                Treino
              """
              params = {'C' : [1, 10, 100, 1000], 'gamma' : ['auto', 2e-1, 2e-2, 2e-3]}
              svm = SVC(probability=True)

              clf = SingleSplitGridSearch(svm, params, test_size=validation_test_size)
              clf.fit(train_X, train_Y)
              print("\n------ Melhor Parametro: ", clf.best_params_)

              """
                Fit Para 5 Impostores
              """
              prob = clf.best_estimator_.predict_proba(test_X_five)    

              l = get_labels(patches, test_Y_five)
              p = get_final_predictions_sumvote(patches, prob)

              f1_five = f1_score(l, p, average='weighted')
              print("-------------- F1_Score --- FIVE: ",f1_five)

              """
                Fit Para TODOS Impostores
              """
              prob = clf.best_estimator_.predict_proba(test_X_all)    

              l = get_labels(patches, test_Y_all)
              p = get_final_predictions_sumvote(patches, prob)

              f1_all = f1_score(l, p, average='weighted')
              print("-------------- F1_Score --- ALL: ",f1_all)

              results[patches]['f1_five'].append(f1_five)
              results[patches]['f1_all'].append(f1_all)

              sys.stdout.flush()

      print("+------------------------------------\n")
      sujeitos[classe_alvo] = results
    return sujeitos
               
      

# Main

In [ ]:
if __name__ == "__main__":
  
  results = run_experiment(0.25, 
      folds, 
      patches, 
      output_proto + "_X.npy", 
      output_proto + "_Y.npy")
  print("results")
  
  pprint.pprint(results2)


Classe:  1
Patch:  1



+--------------Patches: 1 ---- Test_fold: 1

 ------------ len(train_X)
24

 ------------ train_X[:15
[[0.006308 0.001469 0.002217 ... 0.011192 0.002198 0.002878]
 [0.00692  0.00186  0.003133 ... 0.012323 0.003377 0.003892]
 [0.0063   0.001207 0.002054 ... 0.011131 0.002133 0.002457]
 ...
 [0.00362  0.001202 0.001476 ... 0.00614  0.001406 0.001397]
 [0.003871 0.001277 0.001812 ... 0.008811 0.001988 0.002465]
 [0.004492 0.001091 0.001467 ... 0.006474 0.001504 0.001712]]

 ------------ test_Y_five
[50 34  1  7 65  4]

 ------------ test_Y_all
[55 20 24 50 44 69  9 80 59 15 22 29 37 60 25 34 62 48 43 42 71 75 45 70
 13 12 16 32 17 27 30 57  1 14 47  3 51 52 49 18 36 46 61 10 41 35  7 19
  2 65 64 33 68 31 11 66 26 40  4 28 77 39 74 78 54 79 67 63  5 72 56  8
 21 38 73  6 23 58 53 76]



 ----------------- BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB  1

 ================ IMPOSTORES 

 ------------ test_Y_five
[2 2 1 2 2 2]

 ------------ test_Y_all
[2 2 2 2 




+--------------Patches: 3 ---- Test_fold: 2

 ------------ len(train_X)
72

 ------------ train_X[:15
[[0.003632 0.001431 0.001828 ... 0.007421 0.002081 0.002417]
 [0.004277 0.001355 0.001865 ... 0.010717 0.002044 0.002495]
 [0.003574 0.00098  0.00165  ... 0.007758 0.001913 0.002451]
 ...
 [0.005662 0.00184  0.002386 ... 0.008823 0.002677 0.002694]
 [0.003596 0.001021 0.001702 ... 0.006066 0.001571 0.001772]
 [0.004067 0.001096 0.002016 ... 0.007556 0.002051 0.001893]]

 ------------ test_Y_five
[ 1  1  1 76 76 76 65 65 65 71 71 71 70 70 70 10 10 10]

 ------------ test_Y_all
[18 18 18 29 29 29 17 17 17 62 62 62  7  7  7  1  1  1  6  6  6 79 79 79
 31 31 31 43 43 43 59 59 59 21 21 21 64 64 64 35 35 35 75 75 75 78 78 78
 63 63 63  5  5  5 36 36 36 32 32 32 67 67 67  2  2  2 11 11 11 34 34 34
 73 73 73 69 69 69  9  9  9 51 51 51 40 40 40 56 56 56 23 23 23 37 37 37
 19 19 19 28 28 28 25 25 25  8  8  8 54 54 54 76 76 76 30 30 30 65 65 65
 71 71 71 39 39 39 42 42 42 70 70 70 15 15 15 80 


------ Melhor Parametro:  {'C': 10, 'gamma': 0.002}
-------------- F1_Score --- FIVE:  1.0
-------------- F1_Score --- ALL:  0.989543524416136
Patch:  5



+--------------Patches: 5 ---- Test_fold: 1

 ------------ len(train_X)
120

 ------------ train_X[:15
[[0.002742 0.000907 0.001519 ... 0.004122 0.00146  0.001223]
 [0.00248  0.000957 0.001289 ... 0.005312 0.001133 0.00166 ]
 [0.002417 0.00089  0.001064 ... 0.005666 0.00147  0.001334]
 ...
 [0.005337 0.001837 0.002243 ... 0.010481 0.003229 0.002727]
 [0.005859 0.001699 0.002207 ... 0.009374 0.002812 0.002832]
 [0.004726 0.001543 0.0025   ... 0.010331 0.001914 0.003086]]

 ------------ test_Y_five
[44 44 44 44 44 48 48 48 48 48 75 75 75 75 75  5  5  5  5  5  1  1  1  1
  1 71 71 71 71 71]

 ------------ test_Y_all
[63 63 63 63 63 43 43 43 43 43 12 12 12 12 12 20 20 20 20 20 54 54 54 54
 54 49 49 49 49 49 32 32 32 32 32  4  4  4  4  4 33 33 33 33 33 42 42 42
 42 42 56 56 56 56 56 18 18 18 18 18 11 11 11 11 11  9  9  9  9  9  6  6
  6


------ Melhor Parametro:  {'C': 10, 'gamma': 'auto'}
-------------- F1_Score --- FIVE:  1.0
-------------- F1_Score --- ALL:  0.8930555555555557



+--------------Patches: 5 ---- Test_fold: 4

 ------------ len(train_X)
120

 ------------ train_X[:15
[[0.003198 0.001001 0.00118  ... 0.005305 0.001091 0.00133 ]
 [0.002284 0.000712 0.000964 ... 0.004553 0.000934 0.000994]
 [0.004401 0.001148 0.001369 ... 0.006594 0.001384 0.001796]
 ...
 [0.004908 0.001353 0.00266  ... 0.010298 0.002408 0.002637]
 [0.00478  0.001532 0.001624 ... 0.009143 0.001833 0.002019]
 [0.004502 0.001624 0.003434 ... 0.014016 0.00188  0.003504]]

 ------------ test_Y_five
[ 2  2  2  2  2 72 72 72 72 72  1  1  1  1  1 56 56 56 56 56 73 73 73 73
 73 17 17 17 17 17]

 ------------ test_Y_all
[30 30 30 30 30  2  2  2  2  2 79 79 79 79 79 76 76 76 76 76 33 33 33 33
 33 21 21 21 21 21 61 61 61 61 61 80 80 80 80 80 22 22 22 22 22 12 12 12
 12 12 29 29 29 29 29 72 72 72 72 72 43 43 43 43 43 19 19 19 19 19 38 38
 38 38 38 5


------ Melhor Parametro:  {'C': 10, 'gamma': 'auto'}
-------------- F1_Score --- FIVE:  0.7575757575757575
-------------- F1_Score --- ALL:  0.8928082191780822



+--------------Patches: 10 ---- Test_fold: 2

 ------------ len(train_X)
240

 ------------ train_X[:15
[[0.002686 0.000671 0.001007 ... 0.005581 0.001049 0.001007]
 [0.003657 0.001151 0.001315 ... 0.006657 0.001438 0.002055]
 [0.003863 0.001192 0.001644 ... 0.008095 0.00152  0.002055]
 ...
 [0.007682 0.001713 0.003581 ... 0.014949 0.003426 0.004464]
 [0.003997 0.000934 0.001142 ... 0.004152 0.002284 0.001142]
 [0.00268  0.00086  0.001821 ... 0.005411 0.001315 0.00177 ]]

 ------------ test_Y_five
[55 55 55 55 55 55 55 55 55 55 71 71 71 71 71 71 71 71 71 71  1  1  1  1
  1  1  1  1  1  1 68 68 68 68 68 68 68 68 68 68 49 49 49 49 49 49 49 49
 49 49 44 44 44 44 44 44 44 44 44 44]

 ------------ test_Y_all
[45 45 45 45 45 45 45 45 45 45 34 34 34 34 34 34 34 34 34 34 35 35 35 35
 35 35 35 35 35 35 26 26 26 26 26 26 26 26 26 26  

...


------ Melhor Parametro:  {'C': 1, 'gamma': 'auto'}
-------------- F1_Score --- FIVE:  1.0
-------------- F1_Score --- ALL:  0.9444996320824137



+--------------Patches: 10 ---- Test_fold: 4

 ------------ len(train_X)
240

 ------------ train_X[:15
[[0.001553 0.000797 0.001007 ... 0.004658 0.001049 0.000546]
 [0.000699 0.000534 0.000699 ... 0.001685 0.000904 0.000493]
 [0.00152  0.000411 0.000822 ... 0.004356 0.001068 0.000986]
 ...
 [0.005531 0.000858 0.001887 ... 0.00909  0.001501 0.002444]
 [0.00253  0.000772 0.001329 ... 0.004802 0.001201 0.001372]
 [0.004448 0.000881 0.000797 ... 0.005372 0.001846 0.001259]]

 ------------ test_Y_five
[19 19 19 19 19 19 19 19 19 19 26 26 26 26 26 26 26 26 26 26 55 55 55 55
 55 55 55 55 55 55 57 57 57 57 57 57 57 57 57 57 62 62 62 62 62 62 62 62
 62 62 10 10 10 10 10 10 10 10 10 10]

 ------------ test_Y_all
[64 64 64 64 64 64 64 64 64 64 53 53 53 53 53 53 53 53 53 53 18 18 18 18
 18 18 18 18 18 18  3  3  3  3  3  3  3  3  3  3 30 30 30 30 30 30


------ Melhor Parametro:  {'C': 1, 'gamma': 'auto'}
-------------- F1_Score --- FIVE:  1.0
-------------- F1_Score --- ALL:  0.9228603603603605
+------------------------------------

Classe:  20
Patch:  1



+--------------Patches: 1 ---- Test_fold: 1

 ------------ len(train_X)
24

 ------------ train_X[:15
[[0.005035 0.000891 0.001884 ... 0.008019 0.001764 0.002367]
 [0.003369 0.000794 0.001323 ... 0.006168 0.001244 0.001397]
 [0.003967 0.001041 0.001458 ... 0.006954 0.001522 0.001886]
 ...
 [0.004523 0.001326 0.002002 ... 0.006815 0.00216  0.002404]
 [0.003293 0.000811 0.001383 ... 0.006241 0.001272 0.001361]
 [0.00433  0.001211 0.002121 ... 0.009203 0.002297 0.00272 ]]

 ------------ test_Y_five
[44 20 16 79  4 40]

 ------------ test_Y_all
[44 10 46 70 74 37 24 23 20 47 59 78 69 53 35 34  1 55 32 51 11 31 75 80
 18  6 42 30 58 66 65 12 14 73 63 16 71 67  3 36 48 39  5 45 13 49 76 64
 27 15 28 41 72 33 38 77 43  9  2 60 68 29 54 56 57  7 17 21 19 26 61 79
 50  4 22  8 40 52 62 25]




+--------------Patches: 3 ---- Test_fold: 2

 ------------ len(train_X)
72

 ------------ train_X[:15
[[0.004403 0.001303 0.002166 ... 0.00995  0.001964 0.001955]
 [0.0037   0.001178 0.001667 ... 0.00761  0.00158  0.00233 ]
 [0.004532 0.001473 0.0022   ... 0.008766 0.002174 0.002314]
 ...
 [0.003159 0.000915 0.001158 ... 0.004702 0.001086 0.001401]
 [0.002358 0.0007   0.000929 ... 0.004674 0.000872 0.001029]
 [0.003505 0.000937 0.001689 ... 0.00657  0.001547 0.001476]]

 ------------ test_Y_five
[20 20 20 14 14 14  5  5  5 61 61 61 58 58 58  4  4  4]

 ------------ test_Y_all
[67 67 67 36 36 36 11 11 11 35 35 35 20 20 20 55 55 55 73 73 73 70 70 70
 40 40 40 22 22 22  7  7  7  2  2  2 19 19 19  9  9  9 60 60 60  1  1  1
 28 28 28 38 38 38 14 14 14 76 76 76 46 46 46 21 21 21 23 23 23 16 16 16
 41 41 41 57 57 57 39 39 39 29 29 29 17 17 17 25 25 25 69 69 69 62 62 62
  8  8  8 27 27 27 45 45 45 32 32 32 51 51 51 56 56 56 66 66 66 71 71 71
 78 78 78 12 12 12  5  5  5 72 72 72 61 61 61 13 


------ Melhor Parametro:  {'C': 100, 'gamma': 'auto'}
-------------- F1_Score --- FIVE:  1.0
-------------- F1_Score --- ALL:  0.9516447368421053
Patch:  5



+--------------Patches: 5 ---- Test_fold: 1

 ------------ len(train_X)
120

 ------------ train_X[:15
[[0.005518 0.001691 0.001902 ... 0.007631 0.002301 0.001972]
 [0.006288 0.001856 0.001903 ... 0.009189 0.002413 0.002854]
 [0.005504 0.001307 0.002316 ... 0.008922 0.002087 0.002821]
 ...
 [0.008165 0.001445 0.002844 ... 0.011399 0.002752 0.00211 ]
 [0.005801 0.001903 0.002715 ... 0.009259 0.00246  0.003272]
 [0.006892 0.00174  0.003504 ... 0.015292 0.002576 0.003898]]

 ------------ test_Y_five
[20 20 20 20 20 17 17 17 17 17 68 68 68 68 68 39 39 39 39 39 47 47 47 47
 47 71 71 71 71 71]

 ------------ test_Y_all
[25 25 25 25 25 58 58 58 58 58 79 79 79 79 79 63 63 63 63 63 72 72 72 72
 72 70 70 70 70 70 20 20 20 20 20 10 10 10 10 10 17 17 17 17 17  2  2  2
  2  2 69 69 69 69 69 45 45 45 45 45 65 65 65 65 65 22 22 22 22 22 28 28



------ Melhor Parametro:  {'C': 1, 'gamma': 'auto'}
-------------- F1_Score --- FIVE:  1.0
-------------- F1_Score --- ALL:  0.9444996320824137



+--------------Patches: 5 ---- Test_fold: 4

 ------------ len(train_X)
120

 ------------ train_X[:15
[[0.001561 0.000781 0.000616 ... 0.002794 0.000575 0.000596]
 [0.001505 0.000549 0.000712 ... 0.002582 0.000793 0.001261]
 [0.000946 0.000302 0.000604 ... 0.001187 0.000604 0.000584]
 ...
 [0.005877 0.00153  0.001932 ... 0.009117 0.002596 0.002898]
 [0.003823 0.001403 0.001647 ... 0.0061   0.002379 0.00242 ]
 [0.005246 0.001891 0.001972 ... 0.007849 0.002155 0.002196]]

 ------------ test_Y_five
[69 69 69 69 69 62 62 62 62 62 20 20 20 20 20  5  5  5  5  5 65 65 65 65
 65  9  9  9  9  9]

 ------------ test_Y_all
[51 51 51 51 51 49 49 49 49 49 72 72 72 72 72 24 24 24 24 24 35 35 35 35
 35 79 79 79 79 79 54 54 54 54 54 48 48 48 48 48 27 27 27 27 27 58 58 58
 58 58 30 30 30 30 30 39 39 39 39 39  4  4  4  4  4 12 12 12 12 12 11 11
 11 11 11 28

In [30]:
def run():
    features_extraction = ['', 'LPQ_', 'BSIF_', 'VGG_', 'MOBILENET_']
    
    for feature_name in features_extraction:
        print("=========================================== ", feature_name)
        output_proto = features_folder + feature_name +"locutores_Fold%d_%dx1"
        
        results = run_experiment(0.25, 
          folds, 
          patches, 
          output_proto + "_X.npy", 
          output_proto + "_Y.npy")
        print("results")
        if feature_name == '':
            feature_name = 'LBP_'
        np.save(f'resultados_npy/{feature_name}.npy', [results], allow_pickle=True) 

        pprint.pprint(results)

In [ ]:
run()

Classe:  1
Patch:  1



+--------------Patches: 1 ---- Test_fold: 1

 ------------ len(train_X)
24

 ------------ train_X[:15
[[8.07923153e-02 1.76231757e-01 5.33545576e-03 ... 1.17476150e-01
  9.61289275e-03 1.00084865e+00]
 [1.10574603e-01 4.78480548e-01 0.00000000e+00 ... 7.03044294e-04
  6.73402246e-05 1.05038774e+00]
 [0.00000000e+00 5.15592471e-02 0.00000000e+00 ... 5.96504845e-02
  0.00000000e+00 9.13485110e-01]
 ...
 [6.28484692e-03 3.29216048e-02 0.00000000e+00 ... 7.34084006e-03
  0.00000000e+00 8.48851442e-01]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 1.84340984e-01
  3.70139838e-04 1.31059682e+00]
 [9.89145711e-02 0.00000000e+00 0.00000000e+00 ... 3.96137573e-02
  1.89764157e-03 1.15408039e+00]]

 ------------ test_Y_five
[12 14 74 73 53  1]

 ------------ test_Y_all
[16 34 38 77  2 23 31 12 25 72 48 58 10  7 17 29 65 68 13 64 15 56 21 19
 20 45 66 37 24 71 75 14 55 74  9 51  8 49 79 73 42 43  4 63 22 60 35 41
 61  5 46 44 67 11 62 26 53 80 33 52 18 69  6 76 27 59 


------ Melhor Parametro:  {'C': 1, 'gamma': 'auto'}
-------------- F1_Score --- FIVE:  0.7575757575757575
-------------- F1_Score --- ALL:  0.601086956521739



+--------------Patches: 3 ---- Test_fold: 2

 ------------ len(train_X)
72

 ------------ train_X[:15
[[0.14421657 0.3263635  0.         ... 0.00999754 0.         0.66697705]
 [0.         0.14909573 0.         ... 0.01660058 0.         0.81662148]
 [0.         0.6977222  0.         ... 0.         0.         0.53155106]
 ...
 [0.         0.39698926 0.         ... 0.         0.00794478 0.55332083]
 [0.         0.         0.         ... 0.04594019 0.02304436 0.68546927]
 [0.         0.0291526  0.         ... 0.         0.         0.88464552]]

 ------------ test_Y_five
[17 17 17 19 19 19  1  1  1 15 15 15 11 11 11 76 76 76]

 ------------ test_Y_all
[65 65 65 60 60 60 55 55 55 25 25 25 17 17 17 40 40 40 58 58 58 80 80 80
  5  5  5 49 49 49 53 53 53 32 32 32 20 20 20  6  6  6 62 62 62 46 46 46
 43 43 43 61 61 61 30 30 30 77 77 77 


------ Melhor Parametro:  {'C': 1, 'gamma': 'auto'}
-------------- F1_Score --- FIVE:  0.851851851851852
-------------- F1_Score --- ALL:  0.8202592592592592
Patch:  5



+--------------Patches: 5 ---- Test_fold: 1

 ------------ len(train_X)
120

 ------------ train_X[:15
[[0.         0.51964468 0.         ... 0.08629956 0.09683129 0.74219948]
 [0.         0.68818873 0.         ... 0.05933826 0.30937895 0.66075051]
 [0.         0.         0.         ... 0.05223383 0.09753517 1.25222254]
 ...
 [0.00799222 0.13485555 0.         ... 0.11153713 0.02947902 0.75622684]
 [0.         1.67200792 0.         ... 0.         0.1386116  1.30485654]
 [0.36007866 0.         0.         ... 0.0174296  0.28470942 0.91306198]]

 ------------ test_Y_five
[ 1  1  1  1  1 59 59 59 59 59 72 72 72 72 72 14 14 14 14 14 19 19 19 19
 19 57 57 57 57 57]

 ------------ test_Y_all
[ 6  6  6  6  6 23 23 23 23 23 16 16 16 16 16 28 28 28 28 28 73 73 73 73
 73 54 54 54 54 54  7  7  7  7  7 76 76 76 76 76 75 75 75 75 7


------ Melhor Parametro:  {'C': 10, 'gamma': 'auto'}
-------------- F1_Score --- FIVE:  1.0
-------------- F1_Score --- ALL:  0.9516447368421053



+--------------Patches: 5 ---- Test_fold: 4

 ------------ len(train_X)
120

 ------------ train_X[:15
[[0.         0.         0.         ... 0.         0.15013736 1.40557909]
 [0.         0.         0.         ... 0.         0.26275766 0.3314825 ]
 [0.         0.         0.         ... 0.00747231 0.10070002 1.21166193]
 ...
 [0.         0.04477021 0.         ... 0.         0.10175533 1.5450989 ]
 [0.         1.80989122 0.         ... 0.         0.0447984  1.02892399]
 [0.         0.         0.         ... 0.         0.11526905 0.84996241]]

 ------------ test_Y_five
[ 1  1  1  1  1 73 73 73 73 73 67 67 67 67 67 19 19 19 19 19 43 43 43 43
 43 51 51 51 51 51]

 ------------ test_Y_all
[64 64 64 64 64 76 76 76 76 76 59 59 59 59 59  2  2  2  2  2  5  5  5  5
  5 39 39 39 39 39 40 40 40 40 40 50 50 50 50 50 61 61 61 61 61 60 60 60
 60 60 70 70


------ Melhor Parametro:  {'C': 10, 'gamma': 'auto'}
-------------- F1_Score --- FIVE:  0.7575757575757575
-------------- F1_Score --- ALL:  0.9812893081761006



+--------------Patches: 10 ---- Test_fold: 2

 ------------ len(train_X)
240

 ------------ train_X[:15
[[0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 7.70676807e-02
  1.19792148e-02 1.12668559e-01]
 [0.00000000e+00 1.57158166e-01 0.00000000e+00 ... 0.00000000e+00
  5.52550495e-01 5.96148847e-03]
 [3.37272787e+00 1.68771669e-03 0.00000000e+00 ... 5.27507603e-01
  0.00000000e+00 1.29794493e-01]
 ...
 [1.97828591e-01 6.34911180e-01 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 1.63865447e-01]
 [0.00000000e+00 2.54797310e-01 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 2.30895340e-01]
 [3.17146182e-01 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 2.45692387e-01]]

 ------------ test_Y_five
[65 65 65 65 65 65 65 65 65 65 68 68 68 68 68 68 68 68 68 68 55 55 55 55
 55 55 55 55 55 55  1  1  1  1  

...


------ Melhor Parametro:  {'C': 1, 'gamma': 'auto'}
-------------- F1_Score --- FIVE:  0.851851851851852
-------------- F1_Score --- ALL:  0.7374416433239962
Patch:  5



+--------------Patches: 5 ---- Test_fold: 1

 ------------ len(train_X)
120

 ------------ train_X[:15
[[0.12559524 0.         0.         ... 0.16896895 0.05620907 0.81903493]
 [0.16596913 0.         0.         ... 0.26962528 0.13942561 1.97627902]
 [0.         0.         0.         ... 0.0868061  0.02189645 1.35941601]
 ...
 [0.25252685 0.         0.         ... 0.10902754 0.06017206 0.97597617]
 [0.         0.         0.         ... 0.00752461 0.12008006 2.0004847 ]
 [0.         0.         0.         ... 0.         0.19733134 0.83546424]]

 ------------ test_Y_five
[52 52 52 52 52  4  4  4  4  4 66 66 66 66 66  7  7  7  7  7 74 74 74 74
 74 36 36 36 36 36]

 ------------ test_Y_all
[69 69 69 69 69 67 67 67 67 67 22 22 22 22 22 48 48 48 48 48 55 55 55 55
 55 30 30 30 30 30 53 53 53 53 53 50 50 50 50 50 39 39 39 39 3


------ Melhor Parametro:  {'C': 1, 'gamma': 'auto'}
-------------- F1_Score --- FIVE:  1.0
-------------- F1_Score --- ALL:  0.9810897435897437



+--------------Patches: 5 ---- Test_fold: 4

 ------------ len(train_X)
120

 ------------ train_X[:15
[[0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 1.41099012e-02
  2.04740036e-02 1.48799729e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 2.87019871e-02
  4.30771261e-01 1.40887511e+00]
 [0.00000000e+00 1.07877076e-01 0.00000000e+00 ... 1.30423380e-03
  1.90149754e-01 2.11118627e+00]
 ...
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 2.24035680e-02
  4.76134717e-02 1.17382038e+00]
 [0.00000000e+00 1.82841942e-01 0.00000000e+00 ... 2.42016371e-02
  2.46438421e-02 6.53292060e-01]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  6.53492585e-02 1.02666199e+00]]

 ------------ test_Y_five
[50 50 50 50 50 28 28 28 28 28 48 48 48 48 48 60 60 60 60 60  4  4  4  4
  4 37 37 37 37 37]

 ------------ test_Y_all
[40


------ Melhor Parametro:  {'C': 10, 'gamma': 'auto'}
-------------- F1_Score --- FIVE:  0.7575757575757575
-------------- F1_Score --- ALL:  0.975



+--------------Patches: 10 ---- Test_fold: 2

 ------------ len(train_X)
240

 ------------ train_X[:15
[[0.         0.         0.         ... 0.         0.         0.03715849]
 [0.68182087 0.         0.         ... 0.21633807 0.00275086 0.11000148]
 [0.         1.21010768 0.         ... 0.         0.0323152  0.06899403]
 ...
 [0.         0.05160573 0.         ... 0.         0.02788711 0.16377635]
 [0.         0.7772336  0.         ... 0.         0.         0.53243881]
 [0.11897498 0.         0.         ... 0.         0.         0.24369891]]

 ------------ test_Y_five
[46 46 46 46 46 46 46 46 46 46  9  9  9  9  9  9  9  9  9  9  4  4  4  4
  4  4  4  4  4  4 36 36 36 36 36 36 36 36 36 36  7  7  7  7  7  7  7  7
  7  7 31 31 31 31 31 31 31 31 31 31]

 ------------ test_Y_all
[15 15 15 15 15 15 15 15 15 15 46 46 46 46 46 46 46 46 46 46 23 


------ Melhor Parametro:  {'C': 1, 'gamma': 'auto'}
-------------- F1_Score --- FIVE:  1.0
-------------- F1_Score --- ALL:  0.9516447368421053



+--------------Patches: 10 ---- Test_fold: 4

 ------------ len(train_X)
240

 ------------ train_X[:15
[[0.         0.         0.         ... 0.17287584 0.00784292 0.16553091]
 [0.         0.         0.         ... 0.00857555 0.         0.35779554]
 [0.         0.         0.         ... 0.         0.         0.18434143]
 ...
 [0.         0.         0.         ... 0.         0.         0.16349824]
 [0.         0.         0.         ... 0.25146383 0.         0.16098708]
 [0.         0.         0.         ... 0.         0.         0.16349781]]

 ------------ test_Y_five
[ 4  4  4  4  4  4  4  4  4  4 74 74 74 74 74 74 74 74 74 74 75 75 75 75
 75 75 75 75 75 75 52 52 52 52 52 52 52 52 52 52 63 63 63 63 63 63 63 63
 63 63 18 18 18 18 18 18 18 18 18 18]

 ------------ test_Y_all
[79 79 79 79 79 79 79 79 79 79 56 56 56 56 56 56 56 56 56 56 64 64 


------ Melhor Parametro:  {'C': 10, 'gamma': 'auto'}
-------------- F1_Score --- FIVE:  1.0
-------------- F1_Score --- ALL:  0.989543524416136
+------------------------------------

Classe:  5
Patch:  1



+--------------Patches: 1 ---- Test_fold: 1

 ------------ len(train_X)
24

 ------------ train_X[:15
[[1.69498295e-01 6.73374683e-02 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 8.93338740e-01]
 [4.89213802e-02 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  4.97642905e-03 1.63226128e+00]
 [5.22854567e-01 6.68062344e-02 0.00000000e+00 ... 1.33422448e-03
  0.00000000e+00 1.03856289e+00]
 ...
 [2.42568091e-01 3.47168185e-02 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 8.67341757e-01]
 [6.83466122e-02 8.69866461e-04 0.00000000e+00 ... 6.41156882e-02
  1.81158853e-03 1.33625138e+00]
 [0.00000000e+00 1.26519471e-01 0.00000000e+00 ... 0.00000000e+00
  3.16673145e-02 9.38706398e-01]]

 ------------ test_Y_five
[ 5 55 18 67 21  8]

 ------------ test_Y_all
[51 48 75 24  2 2

In [69]:
np.save(f'/content/test.npy', [results2], allow_pickle=True) 

In [70]:
test = np.load('/content/test.npy', allow_pickle=True)

In [12]:
results2

{1: {1: {'f1_five': [0.7575757575757575,
    1.0,
    0.7575757575757575,
    0.7575757575757575,
    0.7575757575757575],
   'f1_all': [0.975,
    0.9228603603603605,
    0.9216666666666666,
    0.975,
    0.9812893081761006]},
  3: {'f1_five': [0.7575757575757575, 0.7575757575757575, 1.0, 1.0, 1.0],
   'f1_all': [0.9001700680272109,
    0.8464285714285715,
    0.9081213307240702,
    0.9081213307240702,
    0.989543524416136]},
  5: {'f1_five': [0.7575757575757575, 1.0, 1.0, 1.0, 1.0],
   'f1_all': [0.949025974025974,
    0.8028056251740463,
    0.8930555555555557,
    0.9588001867413632,
    0.9444996320824137]},
  10: {'f1_five': [0.7575757575757575, 1.0, 1.0, 1.0, 0.7575757575757575],
   'f1_all': [0.8928082191780822,
    0.8537084618019872,
    0.9444996320824137,
    0.9081213307240702,
    0.975]}},
 2: {1: {'f1_five': [1.0, 1.0, 1.0, 1.0, 1.0],
   'f1_all': [0.9444996320824137,
    0.9444996320824137,
    0.9228603603603605,
    0.9516447368421053,
    0.9444996320824137]},
  

In [74]:
test = np.load('/content/test.npy', allow_pickle=True)
test[0][1] # classe 1 
test[0][1][3] # classe 1, patch 3
test[0][1][3]['f1_all'] # classe 1, patch 3, all f1-score

{1: {'f1_all': [0.9286423841059603,
   0.9810897435897437,
   0.9006321839080462,
   0.9621794871794871,
   1.0],
  'f1_five': [0.7575757575757575, 1.0, 1.0, 0.7575757575757575, 1.0]},
 3: {'f1_all': [0.949025974025974,
   0.9001700680272109,
   0.9733870967741935,
   0.9155285190999477,
   0.8697555057857409],
  'f1_five': [0.7575757575757575, 0.7575757575757575, 1.0, 1.0, 1.0]},
 5: {'f1_all': [0.9074324324324324,
   0.9556451612903226,
   0.9301250762660158,
   0.9588001867413632,
   0.8202592592592592],
  'f1_five': [0.7575757575757575, 0.7575757575757575, 1.0, 1.0, 1.0]},
 10: {'f1_all': [0.975,
   0.8455204216073782,
   0.9810897435897437,
   0.8928082191780822,
   0.9286423841059603],
  'f1_five': [0.7575757575757575,
   1.0,
   1.0,
   0.7575757575757575,
   0.7575757575757575]}}

In [46]:
test[1][1]

{'a': [1]}

In [48]:
results2[1]

{1: {'f1_all': [0.9286423841059603,
   0.9810897435897437,
   0.9006321839080462,
   0.9621794871794871,
   1.0],
  'f1_five': [0.7575757575757575, 1.0, 1.0, 0.7575757575757575, 1.0]},
 3: {'f1_all': [0.949025974025974,
   0.9001700680272109,
   0.9733870967741935,
   0.9155285190999477,
   0.8697555057857409],
  'f1_five': [0.7575757575757575, 0.7575757575757575, 1.0, 1.0, 1.0]},
 5: {'f1_all': [0.9074324324324324,
   0.9556451612903226,
   0.9301250762660158,
   0.9588001867413632,
   0.8202592592592592],
  'f1_five': [0.7575757575757575, 0.7575757575757575, 1.0, 1.0, 1.0]},
 10: {'f1_all': [0.975,
   0.8455204216073782,
   0.9810897435897437,
   0.8928082191780822,
   0.9286423841059603],
  'f1_five': [0.7575757575757575,
   1.0,
   1.0,
   0.7575757575757575,
   0.7575757575757575]}}

# Foleis Code

In [ ]:
print("Oi")

In [ ]:
classe_alvo = 5

train_x = np.array(range(1,81))
train_y = np.array(range(1,81))

impostores = list(range(1,81))
impostores.remove(classe_alvo)
np.random.shuffle(impostores)
impostores = impostores[:5]

In [ ]:
m = [train_y == i for i in impostores]

In [ ]:
impostores

[53, 33, 19, 2, 30]

In [ ]:
import functools as ft
mask = ft.reduce(np.logical_or, m)
mask

array([False,  True, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
        True, False, False, False, False, False, False, False, False,
       False, False,  True, False, False,  True, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False,  True, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False])

In [ ]:
train_x[mask]

array([ 2, 19, 30, 33, 53])

In [ ]:
train_y[mask]

array([ 2, 19, 30, 33, 53])

In [ ]:
train_y[ train_y != 2 ] = 5

In [ ]:
train_y

array([5, 2, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5])

In [ ]:
#train_X = np.load("/content/drive/MyDrive/Estudos/2020-1/oRIENTAis/Locutores/Diego/BSIF_locutores_Fold2_3x1_X.npy")
var = "locutores_Fold%d_%dx1"
train_Y = np.load("/content/drive/MyDrive/Estudos/2020-1/oRIENTAis/Locutores/Diego/LPQ_locutores_Fold2_3x1_Y.npy")
train_Y2 = np.load("/content/drive/MyDrive/Estudos/2020-1/oRIENTAis/Locutores/Diego/LPQ_locutores_Fold3_3x1_Y.npy")
train_Y3 = np.load("/content/drive/MyDrive/Estudos/2020-1/oRIENTAis/Locutores/Diego/LPQ_locutores_Fold4_3x1_Y.npy")
train_Y4 = np.load("/content/drive/MyDrive/Estudos/2020-1/oRIENTAis/Locutores/Diego/LPQ_locutores_Fold5_3x1_Y.npy")
#/content/drive/MyDrive

In [ ]:
train_X


In [ ]:
len(train_Y)

240

In [ ]:
train_Y2

In [ ]:
  train_Y_result = np.concatenate([train_Y, train_Y2, train_Y3, train_Y4])

In [ ]:
train_Y_result

array([21, 21, 21, 22, 22, 22, 23, 23, 23, 24, 24, 24,  3,  3,  3, 25, 25,
       25, 26, 26, 26, 27, 27, 27, 28, 28, 28, 29, 29, 29, 30, 30, 30, 31,
       31, 31, 32, 32, 32, 33, 33, 33, 34, 34, 34,  4,  4,  4, 35, 35, 35,
       36, 36, 36, 37, 37, 37, 38, 38, 38, 39, 39, 39, 40, 40, 40,  1,  1,
        1, 41, 41, 41, 42, 42, 42, 43, 43, 43, 44, 44, 44,  5,  5,  5, 45,
       45, 45, 46, 46, 46, 47, 47, 47, 48, 48, 48, 49, 49, 49, 50, 50, 50,
       51, 51, 51, 52, 52, 52, 53, 53, 53, 54, 54, 54,  6,  6,  6, 55, 55,
       55, 56, 56, 56, 57, 57, 57, 58, 58, 58, 59, 59, 59, 60, 60, 60, 61,
       61, 61, 62, 62, 62, 63, 63, 63, 64, 64, 64,  7,  7,  7, 65, 65, 65,
       66, 66, 66, 67, 67, 67, 68, 68, 68, 69, 69, 69, 70, 70, 70, 71, 71,
       71, 72, 72, 72, 73, 73, 73, 74, 74, 74,  8,  8,  8, 75, 75, 75, 76,
       76, 76, 77, 77, 77, 78, 78, 78, 79, 79, 79, 80, 80, 80,  9,  9,  9,
       10, 10, 10, 11, 11, 11, 12, 12, 12, 13, 13, 13, 14, 14, 14,  2,  2,
        2, 15, 15, 15, 16

In [ ]:
proto_x = "/content/drive/MyDrive/Estudos/2020-1/oRIENTAis/Locutores/Diego/locutores_Fold%d_%dx1_X.npy"
proto_y = "/content/drive/MyDrive/Estudos/2020-1/oRIENTAis/Locutores/Diego/locutores_Fold%d_%dx1_Y.npy"
patches = 3
train_folds = [2, 3, 4, 5]

In [ ]:
x = np.load("/content/drive/MyDrive/Estudos/2020-1/oRIENTAis/Locutores/Diego/LPQ_locutores_Fold1_3x1_X.npy")
y = np.load("/content/drive/MyDrive/Estudos/2020-1/oRIENTAis/Locutores/Diego/LPQ_locutores_Fold1_3x1_Y.npy")

In [ ]:
x[0]

array([3.05500e-03, 6.96000e-04, 1.02100e-03, 9.77000e-04, 4.20000e-03,
       6.52000e-04, 5.99000e-04, 6.87000e-04, 1.10900e-03, 1.23300e-03,
       1.90200e-03, 5.46000e-04, 5.81000e-04, 4.84000e-04, 4.40300e-03,
       7.04000e-04, 5.11000e-04, 8.10000e-04, 1.35600e-03, 2.69400e-03,
       1.31200e-03, 7.13000e-04, 4.84000e-04, 2.44800e-03, 1.78700e-03,
       2.60600e-03, 6.25000e-04, 1.03000e-03, 5.28000e-04, 2.41300e-03,
       1.18900e-03, 7.84000e-04, 1.83200e-03, 1.84000e-03, 2.29800e-03,
       4.65800e-03, 7.40000e-04, 3.96000e-04, 8.28000e-04, 1.84900e-03,
       2.06000e-03, 1.16200e-03, 4.75500e-03, 1.31200e-03, 5.99000e-04,
       9.60000e-04, 1.96400e-03, 7.04000e-04, 1.55900e-03, 1.27700e-03,
       6.96000e-04, 9.16000e-04, 6.77100e-03, 1.09200e-03, 7.48000e-04,
       8.63000e-04, 9.07000e-04, 1.30300e-03, 9.95000e-04, 6.96000e-04,
       9.95000e-04, 6.91200e-03, 1.22400e-03, 6.16000e-04, 5.91700e-03,
       2.60600e-03, 1.94600e-03, 1.05700e-03, 1.89300e-03, 7.750

In [ ]:
train_Y_result_2 = np.concatenate([ np.load(proto_y % (f,patches)) for f in train_folds ])

In [ ]:
train_Y_result_2

array([37, 52, 35, 62, 18, 12, 27, 75, 78,  6, 22,  2, 71, 39, 63, 41, 36,
       73, 66, 30, 62, 57,  7, 29,  1, 25, 53, 64, 17, 20, 64, 44, 19, 21,
       10, 31, 42, 19, 31, 69, 77, 59, 25, 26, 45, 25, 53,  8, 33,  1,  5,
       61, 47, 49, 73, 79, 13, 55, 23, 29,  7, 67, 43, 28, 31,  5, 63, 34,
       18, 55, 62, 79, 61, 30, 75, 15, 11, 10, 77, 77, 26, 48,  9, 34, 33,
       12, 46, 51, 49, 80, 43, 33, 65, 32, 63, 27,  3, 74, 49,  6, 68, 72,
       60,  4, 53, 14, 51, 32, 20, 32,  9, 34, 73,  4, 38, 50,  2, 10, 47,
       56, 17, 51, 18, 40, 22, 45, 61, 42, 50, 46, 48, 67, 50, 41, 28, 75,
       54, 23, 56, 78, 65, 27, 15, 11, 80, 52, 26, 48, 60, 58, 68, 64,  6,
       68, 69, 19, 54, 16, 56, 72,  8, 39, 36, 54, 13, 55, 14, 70, 12, 28,
       44, 35, 36, 15, 57, 16,  8, 41, 70, 14, 24, 72, 71, 17, 69, 60, 35,
       11, 37, 42, 74,  4, 16, 71,  1, 67, 46, 45, 38, 38, 59, 43, 47, 57,
       58,  3, 44,  9, 76,  2, 21, 74, 40, 30, 20, 37, 22, 76, 59, 13, 40,
       80, 58, 70,  7, 66

In [ ]:
def unpack_tensor(features, labels, n_patches):

  f = []
  l = []
  for i in features:
      for j in i:
        f.append(j)

  for i in labels:
      for j in i:
        l.append(j)
        
  return f, l

In [ ]:
def get_tensor(features, labels, n_patches):
  inicio= 0
  ty = []
  tx = []
  for i in range(0, len(features), n_patches):
    tx.append(features[inicio: i+2])
    ty.append(labels[inicio: i+2])
    inicio = i+2
  return tx, ty


In [ ]:
tx = np.array( [ ['a' , 'aa' , 'aaa'], ['a' , 'aa' , 'aaa'],  ['b', 'bb', 'bbb' ],  ['b', 'bb', 'bbb' ], ['c', 'cc', 'ccc'], ['c', 'cc', 'ccc'], ['d', 'dd', 'ddd'] , ['d', 'dd', 'ddd'] ] )

ty = np.array(['1', '1', '2', '2', '3', '3', '4', '4'])

In [ ]:
tx[0]

array(['a', 'aa', 'aaa'], dtype='<U3')

In [ ]:
x, y = get_tensor(tx,ty ,2)
X, Y = shuffle(x, y)
x, y = unpack_tensor(X, Y ,2)

In [ ]:
print(x.shape)

(8, 3)


In [ ]:
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>